In [14]:
import os

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from keras.models import load_model
# from keras.utils.vis_utils import plot_model
from tf_explain.core.grad_cam import GradCAM

notebook_path = '/home/tobi/PycharmProjects/Validas/SebastianHPJ/Document Classification/VGG-16/xai'
vgg16_path = '/home/tobi/PycharmProjects/Validas/SebastianHPJ/Document Classification/VGG-16'
os.chdir(vgg16_path)
print(os.getcwd())

import datasets

os.chdir(notebook_path)
print(os.getcwd())

/home/tobi/PycharmProjects/Validas/SebastianHPJ/Document Classification/VGG-16
/home/tobi/PycharmProjects/Validas/SebastianHPJ/Document Classification/VGG-16/xai


In [15]:
# Check if the GPU is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [16]:
# Load the model
model = load_model(os.path.join('..', 'models', 'model.h5'))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 35, 26, 512)    │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 465920)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    59,637,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 74,353,739 (283.64 MB)

 Trainable params: 74,353,737 (283.64 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [17]:
# Print the summary of the VGG16 model
model.layers[0].summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1122, 846, 3)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 1122, 846, 64)  │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 1122, 846, 64)  │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 561, 423, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 561, 423, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 561, 423, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 280, 211, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 280, 211, 256)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 280, 211, 256)  │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 280, 211, 256)  │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 140, 105, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 140, 105, 512)  │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 140, 105, 512)  │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 140, 105, 512)  │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 70, 52, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 70, 52, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 70, 52, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 70, 52, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 35, 26, 512)    │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [19]:
# Load validation dataset
_, validation = datasets.get(data_path=os.path.join('..', '..', '..', 'Daten_sorted'))

Found 107394 files belonging to 9 classes.
Using 85916 files for training.
Using 21478 files for validation.


In [20]:
for batch_images, batch_labels in validation.take(1):
    for image, label in zip(batch_images, batch_labels):
        test_image = image.numpy()
        test_label = label.numpy().astype('uint8')
        break
        
data = ([test_image], None)

2024-06-06 16:03:56.917470: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [21]:
class_index = np.argmax(test_label)
class_index

7

In [22]:
# class_index = np.argmax(test_label)
# print(f'Class: {class_index}')
# plt.imshow(test_image.numpy() / 255)
# plt.tight_layout()
# plt.show()

In [23]:
# test_image.shape

In [24]:
# test_label.shape

In [25]:
# test_image = np.expand_dims(test_image, axis=0)  # Add batch dimension
# test_image.shape

## Grad-CAM

In [26]:
# Instantiation of the explainer
explainer = GradCAM()

# Call to explain() method
output = explainer.explain(
    validation_data=data,
    model=model.layers[0],  # Use only the VGG16 model
    class_index=class_index,
    layer_name='block5_conv3'
)

plt.imshow(output)
plt.tight_layout()
plt.show()

# Save output
explainer.save(output, '.', 'grad_cam.png')

ValueError: When providing `inputs` as a list/tuple, all values in the list/tuple must be KerasTensors. Received: inputs=[[<KerasTensor shape=(None, 1122, 846, 3), dtype=float32, sparse=False, name=input_layer>]] including invalid value [<KerasTensor shape=(None, 1122, 846, 3), dtype=float32, sparse=False, name=input_layer>] of type <class 'list'>